In [10]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [11]:
import os, sys 
%cd ~/nbs/
sys.path.append(os.path.join(os.getcwd(), "orig" ))

%matplotlib inline 
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

/home/ubuntu/nbs


1. Create batches of validation 
2. Create batches of training 
3. Use average of different models and fit it collectively 
4. Fit the batches
5. Run the training on the train and validation data. 
6. Predict on the test 
7. Run prediction on validation set 
8. See what of the predicted on the validation set is correct 
9. Submit tests to backend 

In [12]:
path = 'data/dogscats/'
model_path = path + 'model/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size = 64

In [13]:
training_batches = get_batches(path + 'train', shuffle=False, batch_size=batch_size)
validation_batches = get_batches(path + 'valid', shuffle=False, batch_size=batch_size)


Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [14]:
(val_classes, training_classes, val_labels, training_lables, 
val_filenames, filenames, test_filenames) = get_classes(path)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


# Create an ensemble of different models

1. Train model with last layer all other layer traininable is false
2. Add model with everything but the last layer. This creates an ensemble. 
3. Fine tune just the dense layers of this model. 
4. Add data augmentation, fine tuning dense layers without computation

In [15]:
model = Vgg16().model
print(model.summary())
conv_layers, fc_layers = split_at(model, Convolution2D)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

In [16]:
conv_model = Sequential(conv_layers)
print(conv_model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
                                                                   lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
                                                                   zeropadding2d_14[1][0]           
___________________________________________________________________________________________

convolution2d_26 (Convolution2D) (None, 512, 14, 14)   2359808     zeropadding2d_26[0][0]           
                                                                   zeropadding2d_26[1][0]           
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
____________________________________________________________________________________________________
None


In [17]:
val_predictions = conv_model.predict_generator(validation_batches, validation_batches.nb_sample)
training_predictions = conv_model.predict_generator(training_batches, training_batches.nb_sample)
save_array(model_path + 'valconvlayer_predictions.bc', val_predictions)
save_array(model_path + 'trainconvlayer_predictions.bc', training_predictions)





In [18]:
validation = get_data(path+'valid')
training = get_data(path+'train')
save_array(model_path + 'valid_data.bc', validation)
save_array(model_path + 'training_data.bc', training)

Found 2000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.


In [19]:
model.pop()
model.pop()


In [20]:
# This is the prediction without the last two layers
# which is dropout and dense 
ll_val_predictions = model.predict_generator(validation_batches, validation_batches.nb_sample)
ll_trn_predictions = model.predict_generator(training_batches, training_batches.nb_sample)

save_array(model_path + 'model_val.bc', model_val_predictions)
save_array(model_path + 'model_trn.bc', model_trn_predictions)

ERROR (theano.gof.cmodule): [Errno 12] Cannot allocate memory


OSError: [Errno 12] Cannot allocate memory

In [ ]:
test = get_data(path+'test')
save_array(model_path + 'test_data.bc', test)



In [ ]:
test = load_array(model_path + 'test_data.bc')

## Last layer

In [ ]:
# This is the last last layer. The original model 
# had dropout, and dense but not BatchNormalization
# This is simply adding BatchNormalization in 
def get_last_layer():
    lastLayer = [BatchNormalization(input_shape=(4096,)), 
                Dropout(0.5),
                Dense(2,activation='softmax')]
    return lastLayer
    

In [ ]:
def train_last_layer():
    lastLayers = get_last_layer()
    ll_model = Sequential(lastLayers)
    ll_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    ll_model.optimizer.lr = 1e-5
    ll_model.fit(ll_val_predictions, ll
    